<a href="https://colab.research.google.com/github/Harshithaprudhivi/Team-15-project/blob/main/ECG_Synthesizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wfdb neurokit2 biosppy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 696.5/696.5 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 36.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 w

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import wfdb, neurokit2 as nk
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, TimeDistributed, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import os

# New section

In [ ]:
from google.colab import drive
from pathlib import Path
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
import wfdb
import numpy as np

def convert_all_wfdb_dirs(base_input_dirs, base_output_root):
    for input_dir in base_input_dirs:
        rel_dir_name = os.path.basename(input_dir.rstrip('/'))
        output_dir = os.path.join(base_output_root, f'npy_{rel_dir_name}')
        os.makedirs(output_dir, exist_ok=True)

        for root, _, files in os.walk(input_dir, topdown=True, followlinks=False):
            for file in files:
                if file.endswith('.hea'):
                    record_path = os.path.join(root, file[:-4])
                    rel_path = os.path.relpath(record_path, input_dir)
                    save_path = os.path.join(output_dir, f'{rel_path}.npy')

                    os.makedirs(os.path.dirname(save_path), exist_ok=True)

                    try:
                        print(f"Reading: {record_path}")
                        record = wfdb.rdrecord(record_path)

                        if record.p_signal is None:
                            print(f"Skipping: {record_path} — p_signal is None")
                            continue

                        if record.p_signal.nbytes > 1e9:  # >1GB, you can adjust this
                            print(f"Warning: {record_path} — p_signal too large to save safely")
                            continue

                        print(f"Saving to: {save_path}")
                        np.save(save_path, record.p_signal)
                        print(f"Saved: {save_path}")

                    except Exception as e:
                        print(f"Failed: {record_path} — {e}")


In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# Define paths to both datasets
base_dirs = {
    "100": "/content/drive/MyDrive/PTB-data/npy_records100",
    "500": "/content/drive/MyDrive/PTB-data/npy_records500"
}

In [ ]:

BATCH_SIZE = 1000

In [ ]:
# Final merged dataset
X_all, Y_all = [], []

In [ ]:
def bandpass_filter(ecg): return ecg

In [ ]:
def normalize(ecg):
    max_val = np.max(np.abs(ecg), axis=0, keepdims=True)
    return ecg / np.where(max_val == 0, 1, max_val)

In [ ]:
def extract_input_output(ecg): return ecg[:, :3], ecg  # I, II, V2


In [ ]:

for label, data_folder in base_dirs.items():
    print(f"\n🔍 Starting processing for {label} Hz folder")

    # Gather all .npy files
    file_list = []
    for root, _, files in os.walk(data_folder):
        for f in files:
            if f.endswith('.npy'):
                file_list.append(os.path.join(root, f))
    print(f"✅ Found {len(file_list)} .npy files in {label} Hz folder")

    # Batch processing
    num_batches = (len(file_list) + BATCH_SIZE - 1) // BATCH_SIZE

    for batch_idx in range(num_batches):
        start = batch_idx * BATCH_SIZE
        end = min(start + BATCH_SIZE, len(file_list))
        batch_files = file_list[start:end]

        X_batch, Y_batch = [], []

        print(f"\n🚀 [{label}Hz] Processing batch {batch_idx + 1}/{num_batches}")

        for file_path in tqdm(batch_files):
            try:
                ecg = np.load(file_path)

                if ecg.ndim != 2 or ecg.shape[1] != 12:
                    print(f"⚠️ Skipped: {file_path} (Invalid shape: {ecg.shape})")
                    continue

                ecg = normalize(bandpass_filter(ecg))
                x, y = extract_input_output(ecg)

                X_batch.append(x)
                Y_batch.append(y)

            except Exception as e:
                print(f"❌ Failed to process {file_path}: {e}")

        # Convert to arrays (stay in RAM)
        X_batch = np.array(X_batch)
        Y_batch = np.array(Y_batch)

        print(f"✅ Processed batch {batch_idx + 1}/{num_batches} — Shape: {X_batch.shape}")

        # 🔸 At this point, you can train or analyze on this batch in-memory
        # Example: model.train_on_batch(X_batch, Y_batch)


🔍 Starting processing for 100 Hz folder
✅ Found 21847 .npy files in 100 Hz folder

🚀 [100Hz] Processing batch 1/22


100%|██████████| 1000/1000 [00:50<00:00, 19.78it/s]


✅ Processed batch 1/22 — Shape: (1000, 1000, 3)

🚀 [100Hz] Processing batch 2/22


100%|██████████| 1000/1000 [04:20<00:00,  3.85it/s]


✅ Processed batch 2/22 — Shape: (1000, 1000, 3)

🚀 [100Hz] Processing batch 3/22


...

100%|██████████| 1000/1000 [04:04<00:00,  4.09it/s]


✅ Processed batch 19/22 — Shape: (1000, 1000, 3)

🚀 [100Hz] Processing batch 20/22


100%|██████████| 1000/1000 [04:05<00:00,  4.08it/s]


✅ Processed batch 20/22 — Shape: (1000, 1000, 3)

🚀 [100Hz] Processing batch 21/22


  3%|▎         | 32/1000 [00:07<04:06,  3.93it/s]